In [1]:
#Importing the required libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [2]:
#loading the data
df_tickets = pd.read_csv('/home/stush/Git/traffic_jam_Nairobi/train_revised.csv', low_memory=False)
df_tickets.head()

,ride_id,seat_number,payment_method,payment_receipt,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity
0,1442,15A,Mpesa,UZUEHCBUSO,17-10-17,7:15,Migori,Nairobi,Bus,49
1,5437,14A,Mpesa,TIHLBUSGTE,19-11-17,7:12,Migori,Nairobi,Bus,49
2,5710,8B,Mpesa,EQX8Q5G19O,26-11-17,7:05,Keroka,Nairobi,Bus,49
3,5777,19A,Mpesa,SGP18CL0ME,27-11-17,7:10,Homa Bay,Nairobi,Bus,49
4,5778,11A,Mpesa,BM97HFRGL9,27-11-17,7:12,Migori,Nairobi,Bus,49


In [3]:
df_new_tickets = df_tickets.drop(['seat_number','payment_method','payment_receipt','travel_to'],axis = 1)
df_new_tickets.head()

,ride_id,travel_date,travel_time,travel_from,car_type,max_capacity
0,1442,17-10-17,7:15,Migori,Bus,49
1,5437,19-11-17,7:12,Migori,Bus,49
2,5710,26-11-17,7:05,Keroka,Bus,49
3,5777,27-11-17,7:10,Homa Bay,Bus,49
4,5778,27-11-17,7:12,Migori,Bus,49


In [4]:
group_by_ride_id = df_tickets.groupby(['ride_id']).size().to_frame(name='num_tickets').reset_index()
group_by_ride_id.count()

ride_id        6249
num_tickets    6249
dtype: int64

In [5]:
df_new_tickets.drop_duplicates(inplace=True)
df_new_tickets.count()

ride_id         6249
travel_date     6249
travel_time     6249
travel_from     6249
car_type        6249
max_capacity    6249
dtype: int64

In [6]:
df_combined = pd.merge(group_by_ride_id,df_new_tickets,on='ride_id',how = 'left')
df_combined.head()

,ride_id,num_tickets,travel_date,travel_time,travel_from,car_type,max_capacity
0,1442,1,17-10-17,7:15,Migori,Bus,49
1,5437,1,19-11-17,7:12,Migori,Bus,49
2,5710,1,26-11-17,7:05,Keroka,Bus,49
3,5777,5,27-11-17,7:10,Homa Bay,Bus,49
4,5778,31,27-11-17,7:12,Migori,Bus,49


In [7]:
df_combined.drop(['ride_id'], axis=1, inplace=True) 

In [8]:
df_combined["travel_date"] = pd.to_datetime(df_combined["travel_date"],infer_datetime_format=True)
df_combined[['t_year','t_month']] = df_combined.travel_date.apply(lambda x: pd.Series(x.strftime("%Y,%m").split(",")))
df_combined["travel_date"] = df_combined["travel_date"].dt.dayofweek 
df_new_set = df_combined
df_new_set.head()

,num_tickets,travel_date,travel_time,travel_from,car_type,max_capacity,t_year,t_month
0,1,1,7:15,Migori,Bus,49,2017,10
1,1,6,7:12,Migori,Bus,49,2017,11
2,1,6,7:05,Keroka,Bus,49,2017,11
3,5,0,7:10,Homa Bay,Bus,49,2017,11
4,31,0,7:12,Migori,Bus,49,2017,11


In [9]:
df_new_set['travel_time'] = pd.to_datetime(df_new_set['travel_time'],infer_datetime_format=True)

In [10]:
df_combined[['hour_dept','mins_dept']] = df_new_set.travel_time.apply(lambda x: pd.Series(x.strftime("%-H,%M").split(",")))
df_processed = df_combined
df_processed.head()

,num_tickets,travel_date,travel_time,travel_from,car_type,max_capacity,t_year,t_month,hour_dept,mins_dept
0,1,1,2018-11-25 07:15:00,Migori,Bus,49,2017,10,7,15
1,1,6,2018-11-25 07:12:00,Migori,Bus,49,2017,11,7,12
2,1,6,2018-11-25 07:05:00,Keroka,Bus,49,2017,11,7,05
3,5,0,2018-11-25 07:10:00,Homa Bay,Bus,49,2017,11,7,10
4,31,0,2018-11-25 07:12:00,Migori,Bus,49,2017,11,7,12


In [11]:
df_model=df_processed.drop(['travel_time','mins_dept'],axis = 1)

In [12]:
df_model.head()

,num_tickets,travel_date,travel_from,car_type,max_capacity,t_year,t_month,hour_dept
0,1,1,Migori,Bus,49,2017,10,7
1,1,6,Migori,Bus,49,2017,11,7
2,1,6,Keroka,Bus,49,2017,11,7
3,5,0,Homa Bay,Bus,49,2017,11,7
4,31,0,Migori,Bus,49,2017,11,7


In [13]:
x=df_model

In [14]:
print(pd.get_dummies(df_model['travel_from']).head())

   Awendo  Homa Bay  Kehancha  Kendu Bay  Keroka  Keumbu  Kijauri  Kisii  \
0       0         0         0          0       0       0        0      0   
1       0         0         0          0       0       0        0      0   
2       0         0         0          0       1       0        0      0   
3       0         1         0          0       0       0        0      0   
4       0         0         0          0       0       0        0      0   

   Mbita  Migori  Ndhiwa  Nyachenge  Oyugis  Rodi  Rongo  Sirare  Sori  
0      0       1       0          0       0     0      0       0     0  
1      0       1       0          0       0     0      0       0     0  
2      0       0       0          0       0     0      0       0     0  
3      0       0       0          0       0     0      0       0     0  
4      0       1       0          0       0     0      0       0     0  


In [15]:
x_new=x.drop(['num_tickets'],1)

In [16]:
for col_name in x_new.columns:
        unique_cat = len(x_new[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories".format(
            col_name=col_name,unique_cat=unique_cat))

Feature 'travel_date' has 7 unique categories
Feature 'travel_from' has 17 unique categories
Feature 'car_type' has 2 unique categories
Feature 'max_capacity' has 2 unique categories
Feature 't_year' has 2 unique categories
Feature 't_month' has 12 unique categories
Feature 'hour_dept' has 9 unique categories


In [17]:
todummy = ['max_capacity']

In [18]:
def dummy_fun (dataframe,l):
    for i in l:
        d = pd.get_dummies(dataframe[i],prefix=i,dummy_na=False)
        dataframe = dataframe.drop(i,1)
        dataframe = pd.concat([dataframe,d],axis=1)
        return dataframe

In [19]:
X = dummy_fun(x_new,todummy)
X.head()

,travel_date,travel_from,car_type,t_year,t_month,hour_dept,max_capacity_11,max_capacity_49
0,1,Migori,Bus,2017,10,7,0,1
1,6,Migori,Bus,2017,11,7,0,1
2,6,Keroka,Bus,2017,11,7,0,1
3,0,Homa Bay,Bus,2017,11,7,0,1
4,0,Migori,Bus,2017,11,7,0,1


In [20]:
list_t = ['travel_date']

In [21]:
X_new = dummy_fun(X,list_t)
X_new.head()

,travel_from,car_type,t_year,t_month,hour_dept,max_capacity_11,max_capacity_49,travel_date_0,travel_date_1,travel_date_2,travel_date_3,travel_date_4,travel_date_5,travel_date_6
0,Migori,Bus,2017,10,7,0,1,0,1,0,0,0,0,0
1,Migori,Bus,2017,11,7,0,1,0,0,0,0,0,0,1
2,Keroka,Bus,2017,11,7,0,1,0,0,0,0,0,0,1
3,Homa Bay,Bus,2017,11,7,0,1,1,0,0,0,0,0,0
4,Migori,Bus,2017,11,7,0,1,1,0,0,0,0,0,0


In [22]:
list_r = ['t_year']
X_new_t = dummy_fun(X_new,list_r)
X_new_t.head()

,travel_from,car_type,t_month,hour_dept,max_capacity_11,max_capacity_49,travel_date_0,travel_date_1,travel_date_2,travel_date_3,travel_date_4,travel_date_5,travel_date_6,t_year_2017,t_year_2018
0,Migori,Bus,10,7,0,1,0,1,0,0,0,0,0,1,0
1,Migori,Bus,11,7,0,1,0,0,0,0,0,0,1,1,0
2,Keroka,Bus,11,7,0,1,0,0,0,0,0,0,1,1,0
3,Homa Bay,Bus,11,7,0,1,1,0,0,0,0,0,0,1,0
4,Migori,Bus,11,7,0,1,1,0,0,0,0,0,0,1,0


In [23]:
list_s = ['travel_from']
X_new_s = dummy_fun(X_new_t,list_s)
X_new_s.head()

,car_type,t_month,hour_dept,max_capacity_11,max_capacity_49,travel_date_0,travel_date_1,travel_date_2,travel_date_3,travel_date_4,...,travel_from_Kisii,travel_from_Mbita,travel_from_Migori,travel_from_Ndhiwa,travel_from_Nyachenge,travel_from_Oyugis,travel_from_Rodi,travel_from_Rongo,travel_from_Sirare,travel_from_Sori
0,Bus,10,7,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Bus,11,7,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Bus,11,7,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bus,11,7,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bus,11,7,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [24]:
list_q = ['car_type']
X_new_q = dummy_fun(X_new_s,list_q)
X_new_q.head()

,t_month,hour_dept,max_capacity_11,max_capacity_49,travel_date_0,travel_date_1,travel_date_2,travel_date_3,travel_date_4,travel_date_5,...,travel_from_Migori,travel_from_Ndhiwa,travel_from_Nyachenge,travel_from_Oyugis,travel_from_Rodi,travel_from_Rongo,travel_from_Sirare,travel_from_Sori,car_type_Bus,car_type_shuttle
0,10,7,0,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,11,7,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,11,7,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,11,7,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,11,7,0,1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [25]:
df_set = X_new_q

In [26]:
df_set.isnull().sum().sort_values(ascending=False).head()

car_type_shuttle    0
car_type_Bus        0
hour_dept           0
max_capacity_11     0
max_capacity_49     0
dtype: int64

In [27]:
y_test = x['num_tickets']
x_test = df_set

In [28]:
model = RandomForestRegressor(n_estimators=100, criterion="mae", n_jobs=-1)
model.fit(x_test,y_test)

RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [29]:
preds_train_set = model.predict(x_test)

In [30]:
print (mean_absolute_error(preds_train_set,y_test))

3.267493198911826


In [31]:
from sklearn import ensemble

In [35]:
rfc = RandomForestRegressor()

In [38]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
}

In [39]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(x_test, y_test)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [200, 500], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [40]:
CV_rfc.best_params_

{'max_depth': 8, 'max_features': 'log2', 'n_estimators': 200}

In [42]:
rfc1 = RandomForestRegressor(n_estimators=200, criterion="mae", n_jobs=-1,max_depth = 8,max_features = 'log2')

In [44]:
rfc1.fit(x_test,y_test)

RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=8,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [45]:
preds_set = rfc1.predict(x_test)
print (mean_absolute_error(preds_set,y_test))

4.190940950552089


In [114]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df):
    combos = list(combinations(list(df.columns),2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]